In [1]:
import cv2

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Adjust threshold value in range 80 to 105 based on your light.
bw_threshold = 105

# User message
font = cv2.FONT_HERSHEY_SIMPLEX
org = (30, 30)
weared_mask_font_color = (255, 255, 255)
not_weared_mask_font_color = (0, 0, 255)
thickness = 2
font_scale = 1
weared_mask = "MASK"
not_weared_mask = "No MASK"

# Load image
img = cv2.imread('mask_image.jpg')

# Convert Image into gray
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Convert image in black and white
(thresh, black_and_white) = cv2.threshold(
    gray, bw_threshold, 255, cv2.THRESH_BINARY)

# detect face
faces = face_cascade.detectMultiScale(gray, 1.1, 4)

# Face prediction for black and white
faces_bw = face_cascade.detectMultiScale(black_and_white, 1.1, 4)

if (len(faces) == 0 and len(faces_bw) == 0):
    cv2.putText(img, "No face found...", org, font, font_scale,
                weared_mask_font_color, thickness, cv2.LINE_AA)
else:
    # Draw rectangle on face
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 255, 255), 2)
        roi_gray = gray[y:y + h, x:x + w]
        roi_color = img[y:y + h, x:x + w]

    # Check if face is wearing mask or not
    # This is a simple approach where we check if the lower part of the face is covered or not
    # You can use a machine learning model for a more accurate detection
    if h > w:  # Face is in portrait orientation
        mask_region = gray[int(y+(h/2)):y+h, x:x+w]
    else:  # Face is in landscape orientation
        mask_region = gray[y:y+h, int(x+(w/4)):int(x+w-(w/4))]

    (mask_thresh, mask_bw) = cv2.threshold(
        mask_region, bw_threshold, 255, cv2.THRESH_BINARY)

    if cv2.countNonZero(mask_bw) == 0:
        cv2.putText(img, not_weared_mask, org, font, font_scale,
                    not_weared_mask_font_color, thickness, cv2.LINE_AA)
    else:
        cv2.putText(img, weared_mask, org, font, font_scale,
                    weared_mask_font_color, thickness, cv2.LINE_AA)

# Show image with results
cv2.imshow('Mask Detection', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
